# 📚Libs

In [2]:
import pyspark
import os
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.functions import count, col, when, sum as pyspark_sum
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from functools import reduce


In [2]:
# Definir a variável de ambiente HADOOP_HOME
os.environ["HADOOP_HOME"] = r"C:\spark-3.5.0-bin-hadoop3"

# Adicionar o diretório bin do Hadoop ao PATH
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["HADOOP_HOME"], "bin")

In [3]:
#Iniciando a sessão com spark
spark = (
    SparkSession.builder
    .master('local')
    .appName("Pyspark_02")
    .getOrCreate()
)

# 🎲Importação

In [4]:
# Define o caminho do diretório onde estão os arquivos CSV
diretorio = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\Dados_SINASC'

# Lista para armazenar os DataFrames de cada arquivo CSV
dataframes = []

# Percorre todos os arquivos no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.csv'):  # Verifica se o arquivo é um CSV
        caminho_arquivo = os.path.join(diretorio, arquivo)
        # Lê o arquivo CSV com o separador ';'
        df = spark.read.option("header", "true").csv(caminho_arquivo, sep=';')
        # Remove a coluna "CONTADOR" ou "contador" do DataFrame, se existir
        for coluna in df.columns:
            if coluna.lower() == "contador":
                df = df.drop(coluna)
        dataframes.append(df)

# Define uma função para unir dois DataFrames preservando apenas as colunas de mesmo nome
def merge_dfs(df1, df2):
    # Seleciona as colunas comuns para ambas os DataFrames
    colunas_comuns = [coluna for coluna in df2.columns if coluna in df1.columns]
    # Seleciona as colunas comuns para ambos os DataFrames
    df2 = df2.select(colunas_comuns)
    # Realiza a união dos DataFrames
    return df1.unionByName(df2)

# Aplica a função de união em todos os DataFrames
df_sinasc = reduce(merge_dfs, dataframes)

In [ ]:
#apagar dps, teste apenas como arquivo de 2023
#caminho2 = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\Dados_SINASC\SINASC_2023.csv'
#df_sinasc = spark.read.option("header", "true").csv(caminho2, sep=';')

In [5]:
df_sinasc.show(5)

+------+--------+----------+-------+--------+---------+------+----------+----------+----------+---------+--------+--------+-----+---------+--------+--------+----+------+------+-------+----+--------+----------+---------+----------+----------+---------+-------+----------+----------+----------+---------+----------+----------+---------+----------+----------+----------+----------+--------+----------+----------+----------+----------+---------+----------+----------+----------+----------+----------+---------+----------+----------+----------+--------+----------+--------+--------+----------+
|ORIGEM|CODESTAB|CODMUNNASC|LOCNASC|IDADEMAE|ESTCIVMAE|ESCMAE|CODOCUPMAE|QTDFILVIVO|QTDFILMORT|CODMUNRES|GESTACAO|GRAVIDEZ|PARTO|CONSULTAS|  DTNASC|HORANASC|SEXO|APGAR1|APGAR5|RACACOR|PESO|IDANOMAL|DTCADASTRO|CODANOMAL|NUMEROLOTE|VERSAOSIST|DTRECEBIM|DIFDATA|DTRECORIGA|NATURALMAE|CODMUNNATU|CODUFNATU|ESCMAE2010|SERIESCMAE|DTNASCMAE|RACACORMAE|QTDGESTANT|QTDPARTNOR|QTDPARTCES|IDADEPAI|DTULTMENST|SEMAGESTAC|TP

In [6]:
# Conta o número de linhas no DataFrame
num_linhas = df_sinasc.count()
# Obtém o número de colunas no DataFrame
num_colunas = len(df_sinasc.columns)
# Exibe a quantidade de linhas e colunas
print(f"Quantidade de linhas: {num_linhas}")
print(f"Quantidade de colunas: {num_colunas}")

Quantidade de linhas: 22076634
Quantidade de colunas: 60


In [7]:
df_sinasc.printSchema()

root
 |-- ORIGEM: string (nullable = true)
 |-- CODESTAB: string (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- LOCNASC: string (nullable = true)
 |-- IDADEMAE: string (nullable = true)
 |-- ESTCIVMAE: string (nullable = true)
 |-- ESCMAE: string (nullable = true)
 |-- CODOCUPMAE: string (nullable = true)
 |-- QTDFILVIVO: string (nullable = true)
 |-- QTDFILMORT: string (nullable = true)
 |-- CODMUNRES: string (nullable = true)
 |-- GESTACAO: string (nullable = true)
 |-- GRAVIDEZ: string (nullable = true)
 |-- PARTO: string (nullable = true)
 |-- CONSULTAS: string (nullable = true)
 |-- DTNASC: string (nullable = true)
 |-- HORANASC: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- APGAR1: string (nullable = true)
 |-- APGAR5: string (nullable = true)
 |-- RACACOR: string (nullable = true)
 |-- PESO: string (nullable = true)
 |-- IDANOMAL: string (nullable = true)
 |-- DTCADASTRO: string (nullable = true)
 |-- CODANOMAL: string (nullable = true)
 |--

# 🛠️Transformação

### DTNASC

In [8]:
# Converte a coluna 'DTNASC' para string e preenche com zeros à esquerda
df_sinasc = df_sinasc.withColumn("DTNASC", to_date(lpad(col("DTNASC").cast("string"), 8, '0'), "ddMMyyyy"))

In [9]:
#Cria a coluna de ano
df_sinasc = df_sinasc.withColumn("ANO", year(df_sinasc["DTNASC"]))

# 🔢Contagens

## Total

In [10]:
total = df_sinasc.groupBy("ANO", "CODMUNNASC").count().withColumnRenamed("count", "qt_linhas")
total.printSchema()
total.show(5)

root
 |-- ANO: integer (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- qt_linhas: long (nullable = false)

+----+----------+---------+
| ANO|CODMUNNASC|qt_linhas|
+----+----------+---------+
|2016|    290330|        1|
|2016|    270630|     2754|
|2016|    240800|     6727|
|2016|    293345|       12|
|2016|    260120|     2340|
+----+----------+---------+
only showing top 5 rows



In [11]:
# Conta o número de linhas no DataFrame
num_linhas = total.count()
# Obtém o número de colunas no DataFrame
num_colunas = len(total.columns)
# Exibe a quantidade de linhas e colunas
print(f"Quantidade de linhas: {num_linhas}")
print(f"Quantidade de colunas: {num_colunas}")

Quantidade de linhas: 30724
Quantidade de colunas: 3


In [12]:
total_pd = total.toPandas()
total_pd.to_csv('total_agreg_sinasc.csv', sep=';', index=False)

## Ignorado

In [13]:
#gerando o df
ignorados = df_sinasc.groupBy("ANO", "CODMUNNASC").agg(*[
    pyspark_sum(when(col(c) == 9, 1).otherwise(0)).alias(c)
    for c in ["ESTCIVMAE","ESCMAE","GESTACAO","GRAVIDEZ","PARTO","CONSULTAS","SEXO","IDANOMAL","TPMETESTIM","TPAPRESENT","STTRABPART","STCESPARTO","ESCMAE2010","TPNASCASSI","ESCMAEAGR1"]
])

ignorados.printSchema()
ignorados.show(3)

root
 |-- ANO: integer (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- ESTCIVMAE: long (nullable = true)
 |-- ESCMAE: long (nullable = true)
 |-- GESTACAO: long (nullable = true)
 |-- GRAVIDEZ: long (nullable = true)
 |-- PARTO: long (nullable = true)
 |-- CONSULTAS: long (nullable = true)
 |-- SEXO: long (nullable = true)
 |-- IDANOMAL: long (nullable = true)
 |-- TPMETESTIM: long (nullable = true)
 |-- TPAPRESENT: long (nullable = true)
 |-- STTRABPART: long (nullable = true)
 |-- STCESPARTO: long (nullable = true)
 |-- ESCMAE2010: long (nullable = true)
 |-- TPNASCASSI: long (nullable = true)
 |-- ESCMAEAGR1: long (nullable = true)

+----+----------+---------+------+--------+--------+-----+---------+----+--------+----------+----------+----------+----------+----------+----------+----------+
| ANO|CODMUNNASC|ESTCIVMAE|ESCMAE|GESTACAO|GRAVIDEZ|PARTO|CONSULTAS|SEXO|IDANOMAL|TPMETESTIM|TPAPRESENT|STTRABPART|STCESPARTO|ESCMAE2010|TPNASCASSI|ESCMAEAGR1|
+----+----------+--

In [14]:
#unpivotando colunas
# Lista de colunas a serem mantidas fixas
fixed_cols = ['ANO', 'CODMUNNASC']

# Lista de colunas a serem unpivotadas
ignorados_cols = [col for col in ignorados.columns if col not in fixed_cols]

# Construindo a expressão para unpivot
ignorados_expr = ", ".join([f"'{col}', {col}" for col in ignorados_cols])

# Aplicando o unpivot
ignorados_df = ignorados.selectExpr(fixed_cols + 
                                    [f"stack({len(ignorados_cols)}, {ignorados_expr})"]). \
                          withColumnRenamed("col0", "TIPO_COLUNA"). \
                          withColumn("TIPO_CONTAGEM", lit("ignorado")). \
                          withColumnRenamed("col1", "TIPO_QUANTIDADE")

#reordenando colunas
ignorados_df = ignorados_df.select("ANO", "CODMUNNASC", "TIPO_COLUNA", "TIPO_CONTAGEM", "TIPO_QUANTIDADE")

ignorados_df.printSchema()
ignorados_df.show(3)

root
 |-- ANO: integer (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- TIPO_COLUNA: string (nullable = true)
 |-- TIPO_CONTAGEM: string (nullable = false)
 |-- TIPO_QUANTIDADE: long (nullable = true)

+----+----------+-----------+-------------+---------------+
| ANO|CODMUNNASC|TIPO_COLUNA|TIPO_CONTAGEM|TIPO_QUANTIDADE|
+----+----------+-----------+-------------+---------------+
|2016|    290330|  ESTCIVMAE|     ignorado|              1|
|2016|    290330|     ESCMAE|     ignorado|              1|
|2016|    290330|   GESTACAO|     ignorado|              0|
+----+----------+-----------+-------------+---------------+
only showing top 3 rows



In [ ]:
# Conta o número de linhas no DataFrame
num_linhas = ignorados_df.count()
# Obtém o número de colunas no DataFrame
num_colunas = len(ignorados_df.columns)
# Exibe a quantidade de linhas e colunas
print(f"Quantidade de linhas: {num_linhas}")
print(f"Quantidade de colunas: {num_colunas}")

In [15]:
ignorados_pd = ignorados_df.toPandas()
ignorados_pd.to_csv('ignorados_agreg_sinasc.csv', sep=';', index=False)

## Nulos

In [16]:
nulos = df_sinasc.groupBy("ANO", "CODMUNNASC").agg(*[
    count(when(col(c).isNull(), c)).alias(c) for c in df_sinasc.columns 
    if c not in ["ANO", "CODMUNNASC"]
])

nulos.printSchema()
nulos.show(3)

root
 |-- ANO: integer (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- ORIGEM: long (nullable = false)
 |-- CODESTAB: long (nullable = false)
 |-- LOCNASC: long (nullable = false)
 |-- IDADEMAE: long (nullable = false)
 |-- ESTCIVMAE: long (nullable = false)
 |-- ESCMAE: long (nullable = false)
 |-- CODOCUPMAE: long (nullable = false)
 |-- QTDFILVIVO: long (nullable = false)
 |-- QTDFILMORT: long (nullable = false)
 |-- CODMUNRES: long (nullable = false)
 |-- GESTACAO: long (nullable = false)
 |-- GRAVIDEZ: long (nullable = false)
 |-- PARTO: long (nullable = false)
 |-- CONSULTAS: long (nullable = false)
 |-- DTNASC: long (nullable = false)
 |-- HORANASC: long (nullable = false)
 |-- SEXO: long (nullable = false)
 |-- APGAR1: long (nullable = false)
 |-- APGAR5: long (nullable = false)
 |-- RACACOR: long (nullable = false)
 |-- PESO: long (nullable = false)
 |-- IDANOMAL: long (nullable = false)
 |-- DTCADASTRO: long (nullable = false)
 |-- CODANOMAL: long (nullable =

In [17]:
#unpivotando colunas
# Lista de colunas a serem mantidas fixas
fixed_cols = ['ANO', 'CODMUNNASC']

# Lista de colunas a serem unpivotadas
nulos_cols = [col for col in nulos.columns if col not in fixed_cols]

# Construindo a expressão para unpivot
nulos_expr = ", ".join([f"'{col}', {col}" for col in nulos_cols])

# Aplicando o unpivot
nulos_df = nulos.selectExpr(fixed_cols + 
                                    [f"stack({len(nulos_cols)}, {nulos_expr})"]). \
                          withColumnRenamed("col0", "TIPO_COLUNA"). \
                          withColumn("TIPO_CONTAGEM", lit("nulo")). \
                          withColumnRenamed("col1", "TIPO_QUANTIDADE")

#reordenando colunas
nulos_df = nulos_df.select("ANO", "CODMUNNASC", "TIPO_COLUNA", "TIPO_CONTAGEM", "TIPO_QUANTIDADE")

nulos_df.printSchema()
nulos_df.show(3)

root
 |-- ANO: integer (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- TIPO_COLUNA: string (nullable = true)
 |-- TIPO_CONTAGEM: string (nullable = false)
 |-- TIPO_QUANTIDADE: long (nullable = true)

+----+----------+-----------+-------------+---------------+
| ANO|CODMUNNASC|TIPO_COLUNA|TIPO_CONTAGEM|TIPO_QUANTIDADE|
+----+----------+-----------+-------------+---------------+
|2016|    290330|     ORIGEM|         nulo|              0|
|2016|    290330|   CODESTAB|         nulo|              0|
|2016|    290330|    LOCNASC|         nulo|              0|
+----+----------+-----------+-------------+---------------+
only showing top 3 rows



In [ ]:
# Conta o número de linhas no DataFrame
num_linhas = nulos_df.count()
# Obtém o número de colunas no DataFrame
num_colunas = len(nulos_df.columns)
# Exibe a quantidade de linhas e colunas
print(f"Quantidade de linhas: {num_linhas}")
print(f"Quantidade de colunas: {num_colunas}")

## Nulos e ignorados

In [19]:
# Renomear a coluna "TIPO_QUANTIDADE" para "qtd_nulos" em nulos_df
nulos_df = nulos_df.withColumnRenamed("TIPO_QUANTIDADE", "QTD_NULOS").drop("TIPO_CONTAGEM")

# Renomear a coluna "TIPO_QUANTIDADE" de ignorados_df para "qtd_ignorados"
ignorados_df = ignorados_df.withColumnRenamed("TIPO_QUANTIDADE", "QTD_IGNORADOS")

# Realizar o left join
nulos_ignorados_df = nulos_df.join(ignorados_df.select("ANO", "CODMUNNASC", "TIPO_COLUNA","QTD_IGNORADOS"), ["ANO", "CODMUNNASC", "TIPO_COLUNA"], "left")

#Substituindo o valor zero na coluna de qtd_ignorados e criando coluna da soma
nulos_ignorados_df  = nulos_ignorados_df.fillna(0, subset=["QTD_IGNORADOS"])\
    .withColumn("QTD_NULOS_IGNORADOS", col("qtd_nulos") + col("qtd_ignorados"))

nulos_ignorados_df.printSchema()
nulos_ignorados_df.show(3)


root
 |-- ANO: integer (nullable = true)
 |-- CODMUNNASC: string (nullable = true)
 |-- TIPO_COLUNA: string (nullable = true)
 |-- QTD_NULOS: long (nullable = true)
 |-- QTD_IGNORADOS: long (nullable = true)
 |-- QTD_NULOS_IGNORADOS: long (nullable = true)

+----+----------+-----------+---------+-------------+-------------------+
| ANO|CODMUNNASC|TIPO_COLUNA|QTD_NULOS|QTD_IGNORADOS|QTD_NULOS_IGNORADOS|
+----+----------+-----------+---------+-------------+-------------------+
|2016|    290330|   IDADEMAE|        0|            0|                  0|
|2016|    290330|   CODESTAB|        0|            0|                  0|
|2016|    290330|     ORIGEM|        0|            0|                  0|
+----+----------+-----------+---------+-------------+-------------------+
only showing top 3 rows



In [ ]:
# Conta o número de linhas no DataFrame
num_linhas = nulos_ignorados_df.count()
# Obtém o número de colunas no DataFrame
num_colunas = len(nulos_ignorados_df.columns)
# Exibe a quantidade de linhas e colunas
print(f"Quantidade de linhas: {num_linhas}")
print(f"Quantidade de colunas: {num_colunas}")

In [20]:
nulos_ignorados_pd = nulos_ignorados_df.toPandas()
nulos_ignorados_pd.to_csv('nulos_ignorados_agreg_sinasc.csv', sep=';', index=False)

# 🎲Bases suporte

In [3]:
#intervalo2 = pd.read_excel(url_planilha, sheet_name='emissoes_C02', usecols= 'A:D', nrows=10)
planilha_muni = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\RELATORIO_DTB_BRASIL_MUNICIPIO.xls'

cols = ['UF', 'Nome_UF', 'Município' ,'Código Município Completo', 'Nome_Município']
df_muni = pd.read_excel(planilha_muni, sheet_name='DTB_2022_Municipio', usecols= cols, skiprows=6, nrows=5577)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [4]:
#renomeando colunas
novo_nome = {
    'UF': 'cod_uf',
    'Nome_UF': 'uf',
    'Município': 'cod_muni',
    'Código Município Completo': 'cod_muni_completo',
    'Nome_Município': 'nome_muni'
}

df_muni.rename(columns=novo_nome, inplace=True)

In [5]:
#transformando col de cod para object
col_object1 = ['cod_uf', 'cod_muni', 'cod_muni_completo']
df_muni[col_object1] = df_muni[col_object1].astype(str)    

In [6]:
#transformando chave muni
df_muni['cod_muni'] = df_muni['cod_muni'].astype(str).str.pad(5, side = 'left', fillchar = '0')
#retirando 1 caracter da coluna cod_municompleto
df_muni['cod_muni_completo'] = df_muni['cod_muni_completo'].str.slice(0,6)

In [7]:
#criando coluna de regiao

# mapeia os estados para suas respectivas regiões
regiao_por_estado = {
    'Norte': ['Acre', 'Amapá', 'Amazonas', 'Pará', 'Rondônia', 'Roraima', 'Tocantins'],
    'Nordeste': ['Alagoas', 'Bahia', 'Ceará', 'Maranhão', 'Paraíba', 'Pernambuco', 'Piauí', 'Rio Grande do Norte', 'Sergipe'],
    'Centro-Oeste': ['Distrito Federal', 'Goiás', 'Mato Grosso', 'Mato Grosso do Sul'],
    'Sudeste': ['Espírito Santo', 'Minas Gerais', 'Rio de Janeiro', 'São Paulo'],
    'Sul': ['Paraná', 'Rio Grande do Sul', 'Santa Catarina']
}

# Função para mapear estados para regiões
def mapear_regiao(uf):
    for regiao, estados in regiao_por_estado.items():
        if uf in estados:
            return regiao
    return None  # Retorna None se o estado não for encontrado

#aplica na coluna
df_muni['regiao'] = df_muni['uf'].apply(mapear_regiao)

In [8]:
df_muni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cod_uf             5570 non-null   object
 1   uf                 5570 non-null   object
 2   cod_muni           5570 non-null   object
 3   cod_muni_completo  5570 non-null   object
 4   nome_muni          5570 non-null   object
 5   regiao             5570 non-null   object
dtypes: object(6)
memory usage: 261.2+ KB


In [9]:
df_muni.head()

,cod_uf,uf,cod_muni,cod_muni_completo,nome_muni,regiao
0,11,Rondônia,00015,110001,Alta Floresta D'Oeste,Norte
1,11,Rondônia,00379,110037,Alto Alegre dos Parecis,Norte
2,11,Rondônia,00403,110040,Alto Paraíso,Norte
3,11,Rondônia,00346,110034,Alvorada D'Oeste,Norte
4,11,Rondônia,00023,110002,Ariquemes,Norte


In [10]:
df_muni.to_csv('d_muni.csv', sep = ';', index=False)